# Exp 4b - Hovering with grid set B

In [1]:
import numpy as np
import random
import datetime
import pandas as pd
import json
import sys
from markdown import markdown
import textwrap
from copy import deepcopy

from msdm.domains import GridWorld
from vgc_project import gridutils, utils, sampsat

# Experiment Parameters

In [2]:
EXPERIMENT_CODE_VERSION = "4b"
EXPECTED_TIME = "15 minutes"

### Instructions and training

In [3]:
def create_gridworld_params(tile_array):
    gw = GridWorld(tile_array, absorbing_features=('G', ), initial_features=('S'))

    params = {
        "feature_array": tile_array,
        "init_state": [gw.initial_states[0]['x'], gw.initial_states[0]['y']],
        "absorbing_states":[[s['x'], s['y']] for s in gw.absorbing_states],
    }
    return params

In [4]:
def generate_instructions(default_trialparams, default_taskparams):
    instructions = []
    instructions.extend([
        {"type": "reCAPTCHA"},
        {
            "type": "fullscreen",
            "fullscreen_mode": True
        },
        {
            "type": "CustomInstructions",
            "instructions": markdown(textwrap.dedent("""
                # Instructions
                Thank you for participating in our experiment!

                You will play a game where you control a blue circle on a grid. You can move up, down, left, or right by pressing the __arrow keys__⬆️⬇️⬅️➡️.

                <img src="static/images/bluedotgrid.png" width="150px">

                The <span style='background-color: green;'><b>Green</b></span> tile is the goal 👀. 

                <img src="static/images/green_goal.png" width="150px">

                __Black__ tiles are walls that you cannot pass through ⛔️.

                <br>

                <span style="background-color: cornflowerblue;color:white"><b>Blue</b></span> tiles are obstacles that might change between different rounds. You cannot pass through these either 🚫. 
            """)),
            "timing_post_trial": 1000,
            "continue_wait_time": 5000,
        },
        {
            "trialparams": {
                **default_trialparams,
                "participantStarts": False,
                'flashMaze': False,
                'hoverMode': False,
                'initialText': "   ",
                'hoverText': "   ",
                "navigationText": """
                    Get to the <span style='background-color: green;'>Green</span> goal.
                    You cannot go through <br><span style='background-color: black;color: white'>Black</span> 
                    or <span style='background-color: cornflowerblue; color:white'>Blue</span> tiles.
                """,
                "grid": "practice-1",
                "roundtype": "practice",
            },
            "taskparams": {
                **default_taskparams,
                **create_gridworld_params([
                    'G............',
                    '.............',
                    '.............',
                    '......#......',
                    '......#......',
                    '......#...0..',
                    '.11#######000',
                    '.1....#......',
                    '.1....#......',
                    '......#......',
                    '.............',
                    '.............',
                    '............S'
                ])
            },
            "type": "GridNavigationHoverReveal"
        }
    ])
    
    # instructions about hovering
    instructions.extend([
        {
            "type": "CustomInstructions",
            "instructions": "Great job!",
            "timing_post_trial": 1000,
            "continue_wait_time": 10,
        },
        {
            "type": 'CustomSurvey',
            "preamble": markdown(textwrap.dedent("""
                # Instructions (read carefully!)
                Great! Now, you will be given similar mazes, however, you will not be able
                to see the location of the <span style="background-color: cornflowerblue;color:white"><b>Blue</b></span>
                obstacles while you move.

                Instead, at the beginning, you will be shown a grid with no obstacles like this:

                <img src="static/images/x_start.png" width="300px">

                <br>
                When you click on the red X, your start location (blue circle) and the goal (green tile)
                will appear. The obstacles will all be invisible and never visible simultaneously (but they still block you).

                <br>
                <b>Important:</b> Before you move, you can use your mouse to <b>slowly hover</b> over the maze
                to reveal the obstacles. But once you start moving, <u>the obstacles can no longer be revealed</u>.

                <br>
                You <b>must</b> answer the following questions correctly to continue.
                <hr>
            """)),
            "maxAttempts": 2,
            "questions": [
                {
                  "prompt": "What should you do with the red X?", 
                  "options": ["Avoid it", "Wait for it to change", "Click on it"], 
                  "required": True, 
                  "requireCorrect": True,
                  "correct": "Click on it",
                  "name": "redXCheck",
                  "type": "multiple-choice"
                },
                {
                  "prompt": "When will the obstacles all be visible simultaneously?", 
                  "options": ["Before the red X appears", "After clicking on the red X", "Never"], 
                  "required": True, 
                  "requireCorrect": True,
                  "correct": "Never",
                  "name": "obstaclesNeverVisibleCheck",
                  "type": "multiple-choice"
                },
                {
                  "prompt": "When can you can hover over the obstacles to reveal them?", 
                  "options": ["Any time", "Before moving", "After moving"], 
                  "required": True, 
                  "requireCorrect": True,
                  "correct": "Before moving",
                  "name": "obstaclesHoverCheck",
                  "type": "multiple-choice"
                }
            ]
        },
        {
            "type": "CustomInstructions",
            "instructions": markdown(textwrap.dedent("""
                # Instructions
                Next, you will do some practice rounds.
                <br>
                <br>
                In the first round only, we will show you the obstacles before they disappear.
            """)),
            "timing_post_trial": 100,
            "continue_wait_time": 100,
        },
        {
            "trialparams": {
                **default_trialparams,
                'requireMouseMove': True,
                'flashMaze': True,
                'FLASHMAZE_DURATION_MS': 2000,
                'initialText': "Click on the red X to start.",
                'hoverText': "Hover over the obstacles to reveal them",
                'navigationText': "Navigate to the goal (you can no longer reveal the obstacles)",
                "grid": "practice-2",
                "roundtype": "practice",
            },
            "taskparams": {
                **default_taskparams,
                **create_gridworld_params([
                    'G............',
                    '.............',
                    '.............',
                    '......#......',
                    '......#......',
                    '......#...0..',
                    '.11#######000',
                    '.1....#......',
                    '.1....#......',
                    '......#......',
                    '.............',
                    '.............',
                    '............S'
                ])
            },
            "type": "GridNavigationHoverReveal"
        },
        {
            "trialparams": {
                **default_trialparams,
                'requireMouseMove': True,
                'initialText': "Click on the red X to start.",
                'hoverText': "Hover over the obstacles to reveal them",
                'navigationText': "Navigate to the goal (you can no longer reveal the obstacles)",
                "grid": "practice-3",
                "roundtype": "practice",
            },
            "taskparams": {
                **default_taskparams,
                **create_gridworld_params([
                    '............G',
                    '.............',
                    '.............',
                    '......#......',
                    '......#......',
                    '......#......',
                    '.11#######...',
                    '.1....#......',
                    '.1....#......',
                    '......#......',
                    '......0......',
                    '.....00......',
                    'S.....0......'
                ])
            },
            "type": "GridNavigationHoverReveal"
        },
        {
            "type": "CustomInstructions",
            "instructions": markdown(textwrap.dedent(f"""
                # Instructions
                Great!

                <br>

                Now we can begin the main part of the experiment. You
                will do a series of more complicated mazes in the same format as those
                you just did (click on X to start, then
                hover to reveal obstacles).

                Remember, <u>you can only hover to reveal obstacles before moving</u>!

                <br>

                Good luck!

            """)),
            "timing_post_trial": 1000,
            "continue_wait_time": 5000,
        }
    ])
    return instructions

In [5]:
def generate_posttask():
    return [
        {
            "type": 'CustomSurvey',
            "questions": [
              {
                  "prompt": "Any general comments?",
                  "rows": 5,
                  "columns":50,
                  "required": True,
                  "name": "generalComments",
                  "type": "textbox"
              },
              {"prompt": "Age", "required": True, "name": "age", "type": "textbox"}, 
              {"prompt": "Gender", "required": True, "name": "gender", "type": "textbox"}, 
            ],
        },
        {
            "type": "SaveGlobalStore"
        }
    ]

### Main trial randomization

In [6]:
def sample_block(rng):
    transformations = ('base', 'rot90', 'rot180', 'rot270', 'vflip', 'hflip', 'trans', 'rtrans')
    mazenames = tuple(["grid-12", "grid-13", "grid-14", "grid-15"])
    m = sorted(mazenames, key=lambda m: rng.random())
    t = sorted(transformations + transformations, key=lambda m: rng.random())
    trials = [dict(zip(['grid', 'transformation'], params)) for params in zip(m, t)]
    sampsat.condition(sampsat.no_repeat([t['transformation'] for t in trials]))
    sampsat.condition(sampsat.no_repeat([t['grid'] for t in trials]))
    return trials

def sample_maintrial_params(n_blocks, rng):
    trials = []
    for bi in range(n_blocks):
        block = sampsat.rejection(lambda : sample_block(rng))
        trials.append(block)
    trials = [t for block in trials for t in block]
    grid_trans = [(t['grid'], t['transformation']) for t in trials]
    sampsat.condition(len(grid_trans) == len(set(grid_trans)))
    sampsat.condition(sampsat.has_close_frequencies([(t['grid'], t['transformation']) for t in trials], 1))
    sampsat.condition(sampsat.has_close_frequencies([t['transformation'] for t in trials], 1))
    sampsat.condition(sampsat.no_repeat([t['transformation'] for t in trials]))
    return trials

### Generating conditions

In [7]:
import tqdm
def createGridNavigationHoverReveal(trialparams, taskparams, basegrids):
    trans_grid = gridutils.transformations[trialparams['transformation']](basegrids[trialparams['grid']])
    gw = GridWorld(trans_grid, absorbing_features=('G',), initial_features=('S'))
    params = {
        "trialparams": {
            **trialparams,
            "roundtype": "main",
        },
        "taskparams": {
            **taskparams,
            "feature_array": trans_grid,
            "init_state": [gw.initial_states[0]['x'], gw.initial_states[0]['y']],
            "absorbing_states":[[s['x'], s['y']] for s in gw.absorbing_states],
        },
        "type": "GridNavigationHoverReveal"
    }
    return params

def generate_timelines(default_trialparams, default_taskparams, basegrids, rng):
    timelines = []
    n_trial_seq = 20
    for _ in tqdm.tqdm(range(n_trial_seq)):
        maintrial_params = sampsat.rejection(lambda : sample_maintrial_params(n_blocks=3, rng=rng))   
        maintrial_params = [{**default_trialparams, **t} for t in maintrial_params]
        instructions = generate_instructions(
            default_trialparams,
            default_taskparams
        )
        posttask = generate_posttask()
        timelines.append(
            [{"type": 'fullscreen', "fullscreen_mode": True}] + 
            instructions + 
            [createGridNavigationHoverReveal(
                trialparams={**trialparams, 'round': ti, 'timeline': len(timelines)},
                taskparams=default_taskparams,
                basegrids=basegrids
             ) for ti, trialparams in enumerate(maintrial_params)] +
            posttask + 
            [{"type": 'fullscreen', "fullscreen_mode": False}]
        )
        timelines.append( #reversed
            [{"type": 'fullscreen', "fullscreen_mode": True}] + 
            instructions + 
            [createGridNavigationHoverReveal(
                trialparams={**trialparams, 'round': ti, 'timeline': len(timelines)},
                taskparams=default_taskparams,
                basegrids=basegrids
             ) for ti, trialparams in enumerate(maintrial_params[::-1])] +
            posttask + 
            [{"type": 'fullscreen', "fullscreen_mode": False}]
        )
    return timelines

In [8]:
default_taskparams = {
    "feature_colors": {
        "#": "black",
        "G": "green",
        "S": "white",
        ".": "white",
        **{i: "mediumblue" for i in "0123456"}
    },
    "wall_features": ["#", ] + list("0123456"),
    "show_rewards": False,   
}
default_trialparams = {
    "TILE_SIZE": 40,
    "participantStarts": True,
    'flashMaze': False,
    'hoverMode': True,
    'requireMouseMove': True,
    'hideCursorDuringNav': True,
    'MAX_TIMESTEPS': int(1e10),
    'DWELL_REVEAL_TIME_MS': 25,
    'PREFLASH_DURATION_MS': 0,
    'FLASHMAZE_DURATION_MS': 0,
    'MASK_DURATION_MS': 500,
    'EXPERIMENT_CODE_VERSION': EXPERIMENT_CODE_VERSION
}
basegrids = json.load(open("../mazes/mazes_12-15.json", 'r'))
rng_seed = 422336

timelines = generate_timelines(default_trialparams, default_taskparams, basegrids, rng=random.Random(rng_seed))

100%|██████████| 20/20 [00:00<00:00, 33.81it/s]


### Sanity checks

In [9]:
shape = lambda g: (len(g[0]), len(g))
N_MAIN_TRIALS = 12
GRID_SHAPE = (13, 13)
grid_trans = set([])

for tl in timelines:
    #test correct number of main trials
    assert len([t for t in tl if t.get('trialparams', False) and t['trialparams']['roundtype'] == 'main']) == N_MAIN_TRIALS
    #test all grids are the same size (including practice)
    assert set([shape(t['taskparams']['feature_array']) for t in tl if t.get('taskparams', False)]) == {GRID_SHAPE}
    grid_trans.update([(t['trialparams']['grid'], t['trialparams']['transformation']) 
                       for t in tl if t.get('trialparams', False) and t['trialparams']['roundtype'] == 'main'])

#grids should appear in nearly every translation (at least 6/8)
grid_trans = pd.DataFrame(grid_trans, columns=['grid', 'trans'])
assert all(grid_trans.groupby('grid')['trans'].count() >= 6)

# SAVE EXPERIMENT CONFIGURATIONS

In [10]:
import configparser
import datetime

EXP_CONFIG_DIR = "../../experiment.psiturkapp/static/config/"
EXP_CONFIG_FILE = EXP_CONFIG_DIR+"config.json.zip"
PSITURKAPP_CONFIG = "../../experiment.psiturkapp/config.txt"

#####################
params = {
    "EXPERIMENT_CODE_VERSION": EXPERIMENT_CODE_VERSION,
    "maintrials_seed": rng_seed,
    "expectedTime": EXPECTED_TIME,
    "config_creation_time": str(datetime.datetime.now()),
    "recruitment_platform": "prolific"
}
experiment_config = {
    "timelines": timelines,
    "params": params,
    "preloadImages": [
        "static/images/bluedotgrid.png", 
        "static/images/goalsquare.png",
        "static/images/green_goal.png",
        "static/images/x_start.png"
    ]
}


In [11]:
# UPDATE PSITURK APP
import zipfile
json.dump(experiment_config,
          open(f"config.json", "w"), separators=(',', ':'))
zipfile.ZipFile("config.json.zip", mode="w", compression=zipfile.ZIP_DEFLATED).write("config.json")

config = configparser.ConfigParser()
res = config.read(PSITURKAPP_CONFIG)
assert len(res) > 0
config["Task Parameters"]["experiment_code_version"] = EXPERIMENT_CODE_VERSION
config["Task Parameters"]["num_conds"] = str(len(experiment_config['timelines']))
print(f"{len(experiment_config['timelines'])} conditions")
with open(PSITURKAPP_CONFIG, 'w') as configfile:
    config.write(configfile)
    
import subprocess
process = subprocess.Popen(f"cp config.json.zip {EXP_CONFIG_FILE}".split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(output, error)
process = subprocess.Popen(f"cp config.json.zip {EXP_CONFIG_DIR + '/config-' + EXPERIMENT_CODE_VERSION}.json.zip".split(), stdout=subprocess.PIPE)
output, error = process.communicate()
print(output, error)

40 conditions
b'' None
b'' None
